In [15]:
# Imports
from music21 import *
from IPython.display import display
import zipfile
import pickle
import numpy as np

In [16]:
## Note: Both these data structures are normalized in terms of key
## so they can be used for training 

# Scores list is a list of scores (file names to be read)
path_to_transposed_midis = r'C:\Dev\python\CS4850\TEAM2_LOCAL_DATA\Transposed_Songs'
scores_list = os.listdir(path_to_transposed_midis)

In [17]:
%%time

# Extract notes and chords
# Empty lists of lists for chords and durations
# Each element of the list is a list of chords (or durations) respectively for 
# a given score
chords_ds = [[] for _ in scores_list]   # Holds chords and notes
durations_ds = [[] for _ in scores_list]

num_rests = 0
for ix, score in enumerate(scores_list):

    # Parse the file with music21 
    score = converter.parse(os.path.join(path_to_transposed_midis, score))

    # All notes/chords in the score
    notes_to_parse = score.recurse()

    for element in notes_to_parse:

        if (isinstance(element, note.Note)):

            # Append to lists
            chords_ds[ix].append(element.pitch.nameWithOctave)
            durations_ds[ix].append(element.duration.quarterLength)

        elif (isinstance(element, note.Rest)):
            num_rests += 1

        elif (isinstance(element, chord.Chord)):

            # Append to lists
            chords_ds[ix].append(''.join(str(n) for n in element.pitches))  # .normalOrder attr would work here, too
            durations_ds[ix].append(element.duration.quarterLength)
    
    # Track progress
    print(f'The {ix}th song has been processed of notes, chords, and durations. There are {num_rests} in the song.')

The n= 0 extraction of notes, chords, and durations is complete. 4 in the song.
The n= 1 extraction of notes, chords, and durations is complete. 8 in the song.
The n= 2 extraction of notes, chords, and durations is complete. 11 in the song.
The n= 3 extraction of notes, chords, and durations is complete. 14 in the song.
The n= 4 extraction of notes, chords, and durations is complete. 18 in the song.
The n= 5 extraction of notes, chords, and durations is complete. 31 in the song.
The n= 6 extraction of notes, chords, and durations is complete. 32 in the song.
The n= 7 extraction of notes, chords, and durations is complete. 32 in the song.
The n= 8 extraction of notes, chords, and durations is complete. 78 in the song.
The n= 9 extraction of notes, chords, and durations is complete. 78 in the song.
The n= 10 extraction of notes, chords, and durations is complete. 134 in the song.
The n= 11 extraction of notes, chords, and durations is complete. 138 in the song.
The n= 12 extraction of no

In [27]:
# Chord_ds
chords_ds[10]

['A3E4',
 'A3E4',
 'A4A5',
 'A3E4',
 'A3A4',
 'E3E4',
 'G3G4',
 'A3A4',
 'A3E4',
 'A3E4',
 'A4A5',
 'A3E4',
 'A3A4',
 'E3E4',
 'G3G4',
 'A3A4',
 'A3E4',
 'A3E4',
 'A4A5',
 'A3E4',
 'A3A4',
 'E3E4',
 'G3G4',
 'A3A4',
 'A3E4',
 'A3E4',
 'A4A5',
 'A3E4',
 'A3A4',
 'E3E4',
 'G3G4',
 'A3A4',
 'E5',
 'E-5',
 'D5',
 'C#5',
 'C5',
 'B4',
 'B-4',
 'A3E4',
 'A3E4',
 'A4A5',
 'A3E4',
 'A3A4',
 'E3E4',
 'G3G4',
 'A3A4',
 'A3E4',
 'A3E4',
 'A4A5',
 'A3E4',
 'A3A4',
 'E3E4',
 'G3G4',
 'A3A4',
 'A3E4',
 'A3E4',
 'A4A5',
 'A3E4',
 'A3A4',
 'E3E4',
 'G3G4',
 'A3A4',
 'A3E4',
 'A3E4',
 'A4A5',
 'A3E4',
 'A3A4',
 'E3E4',
 'G3G4',
 'A3A4',
 'E5',
 'E-5',
 'D5',
 'C#5',
 'C5',
 'B4',
 'B-4']

# Encode it

In [30]:
#flattened
flattend_chords = [item for sublist in chords_ds for item in sublist]
flattend_chords.index('E3')

2642

In [13]:
### Encoding map

## Mapping chords to integers
# The chords_ds is a list of lists so to find unique elements (i.e. chord/note combos)
# in the data, I must inspect an 
unique_chords = np.unique([item for sublist in chords_ds for item in sublist])  # Flatten list of lists
chord_to_int = dict(zip(unique_chords, list(range(0, len(unique_chords)))))  # Uncertain

## Map unique durations to integers
unique_durations = np.unique([item for sublist in durations_ds for item in sublist])
duration_to_int = dict(zip(unique_durations, list(range(0, len(unique_durations)))))

# Print number of unique notes and chords
print(len(unique_chords))

# Print number of unique durations
print(len(unique_durations))

### Decoding map
int_to_chord = {integer:chord for chord, integer in chord_to_int.items()}
int_to_duration = {integer:duration for duration, integer in duration_to_int.items()}

1759
26


In [14]:
# Pickle the results
with open('../../pickled_data/pickled_tentatively_transposed_feature_and_encoding_dict', 'wb') as fobj:
    pickle.dump({
        'chords_ds': chords_ds,
        'durations_ds':durations_ds,
        'chord_to_int': chord_to_int,
        'duration_to_int': duration_to_int,
        'int_to_chord': int_to_chord,
        'int_to_duration': int_to_duration,

    }, fobj)